Exploração dos dados

In [11]:
import pandas as pd
import numpy as np

# Carregando o conjunto de dados
data = pd.read_csv('./matches_brasileirao_serie_a_2022.csv')

# verficando a quantidade de instâncias
instances = data.shape[0]
print('Número de instâncias: ' + str(instances))

# verificando a quantidade de atributos
attributes = data.shape[1]
print('Número de atributos: ' + str(attributes))

# Verificando a presença de dados ausentes
print(data.isna().sum())


Número de instâncias: 380
Número de atributos: 26
match_id                    0
stage                       0
date                        0
team_name_home              0
team_name_away              0
team_home_score             0
team_away_score             0
possession_home             0
possession_away             0
total_shots_home            0
total_shots_away            0
shots_on_target_home        0
shots_on_target_away        0
duels_won_home              0
duels_won_away              0
prediction_team_home_win    0
prediction_draw             0
prediction_team_away_win    0
prediction_quantity         0
location                    0
lineup_home                 0
lineup_away                 0
player_names_home           0
player_numbers_home         0
player_names_away           0
player_numbers_away         0
dtype: int64


Criando variáveis

In [13]:
# Criando a variável 'gols_marcados_temporada' para contabilziar a quantidade de gols marcados na tmeporada
data['gols_marcados_temporada'] = data.groupby('team_name_home')['team_home_score'].cumsum()

# Criando a variável 'gols_sofridos_temporada' para contabilziar a quantidade de gols sofridos na tmeporada
data['gols_sofridos_temporada'] = data.groupby('team_name_home')['team_away_score'].cumsum()

print(data[['team_name_home', 'team_home_score', 'gols_marcados_temporada', 'gols_sofridos_temporada']].head())

  team_name_home  team_home_score  gols_marcados_temporada   
0     América MG                1                        1  \
1  RB Bragantino                0                        0   
2  Internacional                3                        3   
3          Goiás                0                        0   
4         Cuiabá                2                        2   

   gols_sofridos_temporada  
0                        1  
1                        1  
2                        0  
3                        4  
4                        1  


Dividindo os dados em um conjunto de treinamento e testes (80% por 20%)

In [14]:
from sklearn.model_selection import train_test_split

# Divisão dos dados em conjunto de treinamento e teste
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Verificando o tamanho dos conjuntos de treinamento e teste
print("Tamanho do conjunto de treinamento:", len(train_data))
print("Tamanho do conjunto de teste:", len(test_data))

Tamanho do conjunto de treinamento: 304
Tamanho do conjunto de teste: 76


Definindo o algotimo de treinamento

In [15]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Definindo as variáveis de entrada (features) e o alvo (target)
features = ['gols_marcados_temporada', 'gols_sofridos_temporada']
target = 'team_home_score'

# Separando as features e o alvo do conjunto de treinamento e teste
X_train = train_data[features]
y_train = train_data[target]
X_test = test_data[features]
y_test = test_data[target]

# Criando e treinando o modelo Decision Tree
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred = model.predict(X_test)

# Calculando a acurácia do modelo
accuracy = accuracy_score(y_test, y_pred)
print("Acurácia do modelo:", accuracy)

Acurácia do modelo: 0.32894736842105265


Validando o modelo

In [25]:
from sklearn.model_selection import cross_val_score

# Aplicando validação cruzada
scores = cross_val_score(model, X_train, y_train, cv=5)

# Exibindo a acurácia média e desvio padrão
print("Acurácia média da validação cruzada:", scores.mean())
print("Desvio padrão da validação cruzada:", scores.std())

Acurácia média da validação cruzada: 0.2827868852459017
Desvio padrão da validação cruzada: 0.045044693980439636


c:\Python311\Lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


In [29]:
# Dados de teste ou novos dados para prever
new_data = test_data 

# Aplicando as mesmas etapas de pré-processamento nas novas entradas
# ...
# Pré-processamento dos novos dados

# Realizando as previsões
new_features = new_data[features]
predictions = model.predict(new_features)

# Exibindo as previsões
print("Previsões:")
for i, pred in enumerate(predictions):
    print("Amostra", i+1, ": Time irá marcar gol" if pred == 1 else "Time não irá marcar gol")

predictions

Previsões:
Amostra 1 Time não irá marcar gol
Amostra 2 Time não irá marcar gol
Amostra 3 : Time irá marcar gol
Amostra 4 : Time irá marcar gol
Amostra 5 Time não irá marcar gol
Amostra 6 : Time irá marcar gol
Amostra 7 Time não irá marcar gol
Amostra 8 Time não irá marcar gol
Amostra 9 Time não irá marcar gol
Amostra 10 : Time irá marcar gol
Amostra 11 Time não irá marcar gol
Amostra 12 Time não irá marcar gol
Amostra 13 : Time irá marcar gol
Amostra 14 : Time irá marcar gol
Amostra 15 Time não irá marcar gol
Amostra 16 Time não irá marcar gol
Amostra 17 Time não irá marcar gol
Amostra 18 Time não irá marcar gol
Amostra 19 Time não irá marcar gol
Amostra 20 : Time irá marcar gol
Amostra 21 Time não irá marcar gol
Amostra 22 Time não irá marcar gol
Amostra 23 Time não irá marcar gol
Amostra 24 : Time irá marcar gol
Amostra 25 Time não irá marcar gol
Amostra 26 Time não irá marcar gol
Amostra 27 : Time irá marcar gol
Amostra 28 : Time irá marcar gol
Amostra 29 : Time irá marcar gol
Amost

array([2, 0, 1, 1, 2, 1, 0, 2, 0, 1, 2, 2, 1, 1, 2, 2, 2, 0, 0, 1, 2, 2,
       2, 1, 2, 2, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 2, 3, 1, 2, 2, 3,
       0, 1, 1, 2, 1, 1, 1, 2, 1, 4, 2, 0, 3, 0, 1, 2, 1, 0, 1, 1, 4, 1,
       2, 0, 2, 1, 2, 0, 0, 2, 1, 0], dtype=int64)